In [1]:
import ezdxf as edf
import math

In [2]:
def calculateLength(e):
    xy = e.dxf.end - e.dxf.start
    length = math.sqrt(xy[0]**2+xy[1]**2)

    return length

def print_entity(e):
    print("start point: %s\n" % e.dxf.start)
    print("end point: %s\n" % e.dxf.end)
    print("length: %s\n" % calculateLength(e))

def getPoints(e):
    # originalLines.append((e.dxf.start, e.dxf.end))
    originalLines.append(e)
    originalPoints.append(e.dxf.start)
    originalPoints.append(e.dxf.end)
    originalPointsVector.append((e.dxf.start.x, e.dxf.start.y))
    originalPointsVector.append((e.dxf.end.x, e.dxf.end.y))


def getPointsInConvex(allPoints, convex):
    innerPoints = list()
    for e in allPoints:
        if edf.math.is_point_in_polygon_2d(e, convex):
            innerPoints.append(e)
    return innerPoints

def getWorkingAreaLines(workingArea, originalLines):
    workingLines = list()
    for point in workingArea:
        for line in originalLines:
            if edf.math.is_point_on_line_2d(point, (line.dxf.start.x, line.dxf.start.y), (line.dxf.end.x, line.dxf.end.y)):
                workingLines.append(line)
    return list(set(workingLines))
    

def constructPolygon(obstacleLines):
    obstaclePoints = list()
    for line in obstacleLines:
        obstaclePoints.append(line.dxf.start)
        obstaclePoints.append(line.dxf.end)
    
    polygon = edf.math.convex_hull_2d(obstaclePoints)
    
    print(polygon)
    return polygon
    
def getObstaclePolygons(innerLines, originalPoints):
    obstacles = list()
    for i in range(0, len(innerLines), 1):
        if i == len(innerLines)-1:
                break
        obstacle = list()
        firstLine = innerLines[i]
        if len(obstacles) > 0:
            start = firstLine.dxf.start
            end = firstLine.dxf.end
            print(obstacles[-1])
            convex = constructPolygon(obstacles[-1])
            if firstLine in obstacles[-1]:
                continue
            elif edf.math.is_point_in_polygon_2d(start, convex) in [0,1] and edf.math.is_point_in_polygon_2d(end, convex) in [0,1]:
                continue
            
            
        obstacle.append(firstLine)
        for j in range(1, len(innerLines), 1):
            secondLine = innerLines[j]
            intersection = edf.math.intersection_line_line_2d((firstLine.dxf.start, firstLine.dxf.end), \
                                                              (secondLine.dxf.start, secondLine.dxf.end))
        
            if intersection in originalPoints:
                obstacle.append(secondLine)
        
        if len(obstacle) > 0:
            obstacles.append(obstacle)
            
    return obstacles

In [5]:
doc = edf.readfile('/home/pi/cadpos/resources/dxfs/Drawing13 - office table test.dxf')
msp = doc.modelspace()

In [6]:
def getEntitiesInLayer(msp):
    entitiesDict = dict()
    for e in msp.query():
        layer = e.dxf.layer
        if layer not in list(entitiesDict.keys()):
            entitiesDict.update({layer: [e]})
        else:
            entitiesDict[layer].append(e)
        
    return entitiesDict

In [11]:
layerDict = getEntitiesInLayer(msp)

In [4]:
originalLines = list()
originalPoints = list()
originalVectorPoints = list()

for e in msp.query('LINE'):
    originalLines.append(e)
    originalVectorPoints.append(e.dxf.start)
    originalVectorPoints.append(e.dxf.end)
    originalPoints.append((e.dxf.start.x, e.dxf.start.y))
    originalPoints.append((e.dxf.end.x, e.dxf.end.y))
    
workingArea = edf.math.convex_hull_2d(originalVectorPoints)
innerPoints = getPointsInConvex(originalVectorPoints, workingArea)

workingLines = getWorkingAreaLines(workingArea, originalLines)
innerLines = [line for line in originalLines if line not in workingLines]
obstacles = getObstaclePolygons(innerLines, originalPoints)

[<class 'ezdxf.entities.line.Line'> LINE(#366)]


ValueError: Convex hull calculation requires 3 or more unique points.

In [ ]:
def getLinesPoints(layer):
    originalLines = list()
    originalPoints = list()
    originalVectorPoints = list()
    
    for e in layer:
        type = e.dxftype()
        if type == ''

In [240]:
layerDict['drawing']

[<class 'ezdxf.entities.lwpolyline.LWPolyline'> LWPOLYLINE(#461),
 <class 'ezdxf.entities.lwpolyline.LWPolyline'> LWPOLYLINE(#462),
 <class 'ezdxf.entities.hatch.Hatch'> HATCH(#46F),
 <class 'ezdxf.entities.hatch.Hatch'> HATCH(#47D),
 <class 'ezdxf.entities.hatch.Hatch'> HATCH(#48F),
 <class 'ezdxf.entities.hatch.Hatch'> HATCH(#499),
 <class 'ezdxf.entities.hatch.Hatch'> HATCH(#4AC),
 <class 'ezdxf.entities.hatch.Hatch'> HATCH(#4BC),
 <class 'ezdxf.entities.hatch.Hatch'> HATCH(#4C6),
 <class 'ezdxf.entities.hatch.Hatch'> HATCH(#4D0),
 <class 'ezdxf.entities.arc.Arc'> ARC(#579),
 <class 'ezdxf.entities.arc.Arc'> ARC(#57B),
 <class 'ezdxf.entities.arc.Arc'> ARC(#59D),
 <class 'ezdxf.entities.arc.Arc'> ARC(#59E),
 <class 'ezdxf.entities.arc.Arc'> ARC(#59F),
 <class 'ezdxf.entities.spline.Spline'> SPLINE(#5AE)]

In [250]:
sp = layerDict['drawing'][-1]
sp.fit_points[0:]

[(11.22084979033385, 16.13652026111853, 0.0),
 (12.49572657985976, 15.44035752534177, 0.0),
 (14.76217401082599, 15.77073982510796, 0.0),
 (12.92068536355856, 15.77073982510796, 0.0),
 (11.22084979033385, 16.13652026111853, 0.0)]

In [206]:
sp.fit_points[0:]

[(17.77392841720342, 13.79340337257891, 0.0),
 (20.13427298937921, 9.074732233770135, 0.0),
 (16.75323892661063, 9.074732233770135, 0.0),
 (14.67996333625405, 14.14411545494835, 0.0),
 (17.77392841720342, 13.79340337257891, 0.0)]

In [205]:
edf.lldxf.const

<module 'ezdxf.lldxf.const' from '/Users/lawrence/Desktop/ezdxf/.venv/lib/python3.8/site-packages/ezdxf/lldxf/const.py'>

In [277]:
def getSPFitLines(spline):
    fitLines = list()
    spPoints = spline.fit_points
    for i in range(len(spPoints)):
        startPoint = spPoints[i]
        if i == len(spPoints)-1:
            endPoint = spPoints[0]
        else:
            endPoint = spPoints[i+1]
        fitLines.append((edf.math.Vec2(startPoint[0], startPoint[1]), edf.math.Vec2(endPoint[0], endPoint[1])))
    return fitLines

In [278]:
spFitLines = getSPFitLines(sp)

In [264]:
for point in sp.fit_points:
    print(point)

(11.22084979033385, 16.13652026111853, 0.0)
(12.49572657985976, 15.44035752534177, 0.0)
(14.76217401082599, 15.77073982510796, 0.0)
(12.92068536355856, 15.77073982510796, 0.0)
(11.22084979033385, 16.13652026111853, 0.0)


In [ ]:
def isInSPLine(fitLines, testingPoint, spline):
    isIntersection = list()
    for point in spline.fit_points:
        cuttingLine = (testingPoint, edf.math.Vec2(point[0], point[1]))
        intersection = 0
        for lines in fitLines:
            intersectionPoint = edf.math.intersection_line_line_2d(cuttingLine, (lines[0], lines[1]))
            if intersectionPoint:
                intersection += 1
        if intersection % 2 == 1:
            isIntersection.append(True)
        
        if len([i for i in isIntersection if i == True]) == 3:
            return True
    
    return False

In [229]:
ezdxf.math.is_point_in_polygon_2d(edf.math.Vec2(sp.fit_points[0][0], sp.fit_points[0][1]), testPolygon)

0

In [280]:
edf.math.intersection_line_line_2d((spFitLines[0][0], spFitLines[0][1]), \
                                                              (spFitLines[1][0], spFitLines[1][1]))

Vec2(12.49572657985976, 15.440357525341776)

In [258]:
edf.math.Vector

ezdxf.math.vector.Vector

In [261]:
originalLines[0].dxf.end

Vector(27.51999852234022, 17.50996849717288, 0.0)

In [ ]:
def isInObstacles(obstaclePolygon, point):
    return edf.math.is_point_in_polygon_2d(point, obstaclePolygon)

In [5]:
import numpy as np

In [54]:
def discretizeWorkingArea(workingArea):
    allX = [i[0] for i in workingArea]
    allY = [i[1] for i in workingArea]
    
    leftX = min(allX)
    rightX = max(allX)
    bottomY = min(allY)
    topY = max(allY)
    
    xyPairs = np.mgrid[leftX:rightX+0.01:, bottomY:topY+0.01:0.01].reshape(2,-1).T
    discretePoints = [edf.math.Vec2(i[0], i[1]) for i in xyPairs]

    return discretePoints            

In [56]:
dps = discretizeWorkingArea(workingArea)

In [290]:
workingArea

[Vec2(12.4858778935436, 9.960175177181172),
 Vec2(12.4858778935436, 17.50996849717288),
 Vec2(27.51999852234022, 17.50996849717288),
 Vec2(27.51999852234022, 9.96017517718117)]